In [1]:
# Import libraries
# Model design
import agentpy as ap
# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import IPython
import random
from time import time

In [2]:
# Define parameters
parameters = {
    'm': 20, # Height of the grid
    'n': 30, # Length of the grid
    'k': 10, # Number of robots
    'tMax': 100 # Max running time
}

In [3]:
class IntersectionModel(ap.Model):
    
    def setup(self):
        # Generate random starting positions
        random.seed(time())
        self.pos=[(1, round(self.p.n/2)-1), (self.p.m-2, round(self.p.n/2)), (round(self.p.m/2), 1), (round(self.p.m/2)-1, self.p.n-2)]
        for i in range(0,self.p.k):
            self.pos.append(self.pos[random.randint(0,3)])
        # Create agents (Carss)
        cars = self.agents = ap.AgentList(self, self.p.k)
        # Create grid (Room)
        self.street = ap.Grid(self, (self.p.m, self.p.n), track_empty=True)
        self.street.add_agents(cars, positions=self.pos[4:14], empty=True)
        # Dynamic variable conditions
        self.agents.condition = 0
        self.agents.orientation = 'x'
        
    def step(self):
        # Run for every car
        for i in self.agents:
            # On Start (Set orientation)
            if(self.street.positions[i] == self.pos[0]):
                i.orientation = 'n' 
            if(self.street.positions[i] == self.pos[1]):
                i.orientation = 's'
            if(self.street.positions[i] == self.pos[2]):
                i.orientation = 'w'
            if(self.street.positions[i] == self.pos[3]):
                i.orientation = 'e'
            # On End (Reset position)
            if(self.street.positions[i] == (self.p.m-2, round(self.p.n/2)-1) # Final 'n' position
              or self.street.positions[i] == (1, round(self.p.n/2)) # Final 's' position
              or self.street.positions[i] == (round(self.p.m/2), self.p.n-2) # Final 'w' position
              or self.street.positions[i] == (round(self.p.m/2)-1, 1)): # Final 'e' position
                i.orientation = 'x'
                self.street.move_to(i, self.pos[random.randint(0,3)])
            # Control movement
            if(i.orientation == 'n' 
                and not self.street.agents[self.street.positions[i][0]+1, self.street.positions[i][1]].condition == True
                and not self.street.agents[self.street.positions[i][0]+2, self.street.positions[i][1]].condition == True):
                    self.street.move_by(i, (1, 0))
            if(i.orientation == 's'
               and not self.street.agents[self.street.positions[i][0]-1, self.street.positions[i][1]].condition == True
               and not self.street.agents[self.street.positions[i][0]-2, self.street.positions[i][1]].condition == True):
                    self.street.move_by(i, (-1, 0))
            if(i.orientation == 'w'
               and not self.street.agents[self.street.positions[i][0], self.street.positions[i][1]+1].condition == True
               and not self.street.agents[self.street.positions[i][0], self.street.positions[i][1]+2].condition == True):
                self.street.move_by(i, (0, 1))
            if(i.orientation == 'e'
               and not self.street.agents[self.street.positions[i][0], self.street.positions[i][1]-1].condition == True
               and not self.street.agents[self.street.positions[i][0], self.street.positions[i][1]-2].condition == True):
                self.street.move_by(i, (0, -1))           
        if self.t == self.p.tMax:
            self.stop()

    def end(self):
        # Document a measure at the end of the simulation
        return

In [4]:
# Create a single-run animation with custom colors
def animation_plot(model, ax):
    attr_grid = model.street.attr_grid('condition')
    color_dict = {0:'#ff2800', None:'#525257'}
    ap.gridplot(attr_grid, ax=ax, color_dict=color_dict, convert=True)
    ax.set_title(f"Intersection simulation\n"
                 f"Time-step: {model.t}")
fig, ax = plt.subplots()
model = IntersectionModel(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml(fps=15))